In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
from warnings import filterwarnings
filterwarnings('ignore')
import json

In [2]:
card = pd.read_csv('cards_data.csv')
tr = pd.read_csv('transactions_data.csv')
users = pd.read_csv('users_data.csv')

In [3]:
with open('mcc_codes.json', 'r') as file:
    codes = json.load(file)

with open('train_fraud_labels.json', 'r') as file:
    fraud = json.load(file)

In [4]:
fraud_id = list(fraud['target'].keys())
fraud_status = list(fraud['target'].values())

df1 = pd.DataFrame({"id": fraud_id, 'Status': fraud_status})
df1.head()

,id,Status
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No


In [5]:
code = list(codes.keys())
name = list(codes.values())

df2 = pd.DataFrame({'mcc': code, 'Name': name})
df2.head()

,mcc,Name
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [6]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


In [7]:
for i in card.select_dtypes(include='object').columns:
    print(card[i].value_counts())
    print('****'*20)

card_brand
Mastercard    3209
Visa          2326
Amex           402
Discover       209
Name: count, dtype: int64
********************************************************************************
card_type
Debit              3511
Credit             2057
Debit (Prepaid)     578
Name: count, dtype: int64
********************************************************************************
expires
02/2020    377
01/2020    130
03/2020     93
10/2024     93
03/2024     93
          ... 
01/2002      1
07/1997      1
07/2005      1
06/2001      1
05/2006      1
Name: count, Length: 259, dtype: int64
********************************************************************************
has_chip
YES    5500
NO      646
Name: count, dtype: int64
********************************************************************************
credit_limit
$0        31
$8000     25
$8700     25
$8500     24
$9300     24
          ..
$11696     1
$10198     1
$5716      1
$16872     1
$28074     1
Name: count, Length: 3654, d

In [8]:
card['credit_limit'] = card['credit_limit'].str.replace('$', '')

In [9]:
card['credit_limit'] = card['credit_limit'].astype(int)

In [10]:
card['acct_open_date'] = pd.to_datetime(card['acct_open_date'])
card['expires'] =pd.to_datetime(card['expires'])

In [11]:
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   int32         
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   object        
dtypes: 

In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


In [13]:
for i in users.select_dtypes(include='object').columns:
    print(users[i].value_counts())
    print('****'*20)

gender
Female    1016
Male       984
Name: count, dtype: int64
********************************************************************************
address
506 Washington Lane      2
462 Rose Lane            1
4480 Hillside Avenue     1
876 Main Avenue          1
68 Sixth Avenue          1
                        ..
499 Lafayette Lane       1
388 Third Street         1
512 Tenth Street         1
6840 North Lane          1
472 Ocean View Street    1
Name: count, Length: 1999, dtype: int64
********************************************************************************
per_capita_income
$0        15
$19382     4
$13739     3
$40364     3
$21992     3
          ..
$17537     1
$17584     1
$24331     1
$15138     1
$32325     1
Name: count, Length: 1754, dtype: int64
********************************************************************************
yearly_income
$44128    3
$44976    2
$34954    2
$42806    2
$49967    2
         ..
$36081    1
$72470    1
$553      1
$24884    1
$65909    1
Na

In [14]:
users['per_capita_income'] = users['per_capita_income'].str.replace('$', '')

In [15]:
users['per_capita_income'] = users['per_capita_income'].astype(int)

In [16]:
users['yearly_income'] = users['yearly_income'].str.replace('$', '')

In [17]:
users['yearly_income'] = users['yearly_income'].astype(int)

In [18]:
users['total_debt'] = users['total_debt'].str.replace('$', '')

In [19]:
users['total_debt'] = users['total_debt'].astype(int)

In [20]:
users_card = pd.merge(users, card, on = 'id', how = 'inner')

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      object
 1   Status  object
dtypes: object(2)
memory usage: 136.0+ MB


In [22]:
df1['id'] = df1['id'].astype(int)

In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mcc     109 non-null    object
 1   Name    109 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [24]:
df2['mcc'] = df2['mcc'].astype(int)

In [25]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [26]:
for i in tr.select_dtypes(include='object').columns:
    print(tr[i].value_counts())
    print('****'*20)

date
2011-06-09 12:46:00    18
2016-03-03 11:42:00    18
2018-07-18 09:38:00    17
2015-11-08 09:03:00    17
2018-08-18 11:43:00    17
                       ..
2015-03-20 04:30:00     1
2015-03-20 04:19:00     1
2015-03-20 04:13:00     1
2015-03-20 04:11:00     1
2019-10-31 23:59:00     1
Name: count, Length: 4136496, dtype: int64
********************************************************************************
amount
$80.00     132115
$100.00    128867
$60.00     101821
$120.00     81083
$40.00      56856
            ...  
$605.45         1
$495.35         1
$852.48         1
$944.73         1
$694.30         1
Name: count, Length: 81161, dtype: int64
********************************************************************************
use_chip
Swipe Transaction     6967185
Chip Transaction      4780818
Online Transaction    1557912
Name: count, dtype: int64
********************************************************************************
merchant_city
ONLINE          1563700
Houston       

In [27]:
tr['amount'] = tr['amount'].str.replace('$', '')

In [28]:
tr['amount'] = tr['amount'].astype(float)

In [29]:
tr.isnull().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1563700
zip                1652706
mcc                      0
errors            13094522
dtype: int64

In [30]:
tr_df1 = pd.merge(tr, df1, on='id', how='inner')

In [31]:
tr_df1_df2 = pd.merge(tr_df1, df2, on = 'mcc', how = 'inner')

In [32]:
tr_df1_df2.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,Status,Name
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,No,Miscellaneous Food Stores
1,7475340,2010-01-01 00:26:00,1556,2972,39.63,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN,No,Miscellaneous Food Stores
2,7475366,2010-01-01 01:03:00,1703,2969,78.00,Swipe Transaction,43293,Grand Junction,CO,81506.0,5499,NaN,No,Miscellaneous Food Stores
3,7475379,2010-01-01 01:22:00,986,4755,1.85,Swipe Transaction,14528,Garden City,NY,11530.0,5499,NaN,No,Miscellaneous Food Stores
4,7475380,2010-01-01 01:23:00,1703,2969,71.72,Swipe Transaction,43293,Grand Junction,CO,81506.0,5499,NaN,No,Miscellaneous Food Stores


In [33]:
tr_df1_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 14 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          float64
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
 12  Status          object 
 13  Name            object 
dtypes: float64(2), int64(5), object(7)
memory usage: 952.2+ MB


In [34]:
users_card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     2000 non-null   int64         
 1   current_age            2000 non-null   int64         
 2   retirement_age         2000 non-null   int64         
 3   birth_year             2000 non-null   int64         
 4   birth_month            2000 non-null   int64         
 5   gender                 2000 non-null   object        
 6   address                2000 non-null   object        
 7   latitude               2000 non-null   float64       
 8   longitude              2000 non-null   float64       
 9   per_capita_income      2000 non-null   int32         
 10  yearly_income          2000 non-null   int32         
 11  total_debt             2000 non-null   int32         
 12  credit_score           2000 non-null   int64         
 13  num

In [35]:
df = pd.merge(tr_df1_df2,users_card, on='id', how='inner')

In [36]:
df.info()#id's are differet :(

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     0 non-null      int64         
 1   date                   0 non-null      object        
 2   client_id_x            0 non-null      int64         
 3   card_id                0 non-null      int64         
 4   amount                 0 non-null      float64       
 5   use_chip               0 non-null      object        
 6   merchant_id            0 non-null      int64         
 7   merchant_city          0 non-null      object        
 8   merchant_state         0 non-null      object        
 9   zip                    0 non-null      float64       
 10  mcc                    0 non-null      int64         
 11  errors                 0 non-null      object        
 12  Status                 0 non-null      object        
 13  Name             

In [37]:
for i in tr_df1_df2.select_dtypes(include='object').columns:
    print(tr_df1_df2[i].value_counts())
    print('****'*20)

date
2013-01-07 12:53:00    14
2018-11-23 07:16:00    14
2016-09-15 11:06:00    13
2017-07-25 11:13:00    13
2015-11-08 09:03:00    13
                       ..
2013-11-17 20:33:00     1
2013-11-17 18:44:00     1
2013-11-17 17:02:00     1
2013-11-17 15:43:00     1
2019-08-07 05:23:00     1
Name: count, Length: 3680988, dtype: int64
********************************************************************************
use_chip
Swipe Transaction     4668212
Chip Transaction      3202776
Online Transaction    1043975
Name: count, dtype: int64
********************************************************************************
merchant_city
ONLINE         1047865
Houston          98502
Miami            58680
Brooklyn         56173
Los Angeles      54969
                ...   
Delco                1
Geff                 1
What Cheer           1
Guy                  1
Long Bottom          1
Name: count, Length: 12173, dtype: int64
***********************************************************************

In [38]:
df = tr_df1_df2.copy()

In [39]:
df=df.drop(["date"],axis=1)

In [40]:
df = pd.get_dummies(df, columns=['use_chip'])

In [41]:
df=df.drop(["merchant_city"],axis=1)

In [42]:
df=df.drop(["merchant_state"],axis=1)

In [43]:
df = pd.get_dummies(df, columns=['errors'])

In [44]:
df = pd.get_dummies(df, columns=['Status'])

In [45]:
df=df.drop(["Status_No"],axis=1)

In [46]:
df = pd.get_dummies(df, columns=['Name'])

In [47]:
corr_matrix=df.corr()
corr_matrix["Status_Yes"].sort_values()

use_chip_Swipe Transaction                -0.032572
mcc                                       -0.015118
Name_Miscellaneous Food Stores            -0.012380
Name_Service Stations                     -0.011829
Name_Grocery Stores, Supermarkets         -0.010468
                                             ...   
Name_Music Stores - Musical Instruments    0.045934
Name_Electronics Stores                    0.050007
use_chip_Online Transaction                0.065160
Name_Cruise Lines                          0.085867
Status_Yes                                 1.000000
Name: Status_Yes, Length: 141, dtype: float64

In [48]:
sorted_corr=corr_matrix["Status_Yes"].sort_values()

In [49]:
filtered_corr = sorted_corr[(sorted_corr > 0.0085) | (sorted_corr < -0.0085)]
print(filtered_corr)

use_chip_Swipe Transaction                               -0.032572
mcc                                                      -0.015118
Name_Miscellaneous Food Stores                           -0.012380
Name_Service Stations                                    -0.011829
Name_Grocery Stores, Supermarkets                        -0.010468
use_chip_Chip Transaction                                -0.009763
Name_Eating Places and Restaurants                       -0.009698
Name_Tolls and Bridge Fees                               -0.008942
Name_Automotive Parts and Accessories Stores              0.008509
Name_Steel Drums and Barrels                              0.008754
Name_Cosmetic Stores                                      0.008815
Name_Coated and Laminated Products                        0.008929
Name_Non-Precious Metal Services                          0.009483
Name_Industrial Equipment and Supplies                    0.009536
Name_Bus Lines                                            0.00

In [50]:
filtered_columns = filtered_corr.index.tolist()

df= df[filtered_columns]

df.head()

,use_chip_Swipe Transaction,mcc,Name_Miscellaneous Food Stores,Name_Service Stations,"Name_Grocery Stores, Supermarkets",use_chip_Chip Transaction,Name_Eating Places and Restaurants,Name_Tolls and Bridge Fees,Name_Automotive Parts and Accessories Stores,Name_Steel Drums and Barrels,...,Name_Department Stores,"Name_Furniture, Home Furnishings, and Equipment Stores",amount,Name_Precious Stones and Metals,"Name_Computers, Computer Peripheral Equipment",Name_Music Stores - Musical Instruments,Name_Electronics Stores,use_chip_Online Transaction,Name_Cruise Lines,Status_Yes
0,True,5499,True,False,False,False,False,False,False,False,...,False,False,-77.00,False,False,False,False,False,False,False
1,True,5499,True,False,False,False,False,False,False,False,...,False,False,39.63,False,False,False,False,False,False,False
2,True,5499,True,False,False,False,False,False,False,False,...,False,False,78.00,False,False,False,False,False,False,False
3,True,5499,True,False,False,False,False,False,False,False,...,False,False,1.85,False,False,False,False,False,False,False
4,True,5499,True,False,False,False,False,False,False,False,...,False,False,71.72,False,False,False,False,False,False,False


In [51]:
X = df.drop(columns=['Status_Yes'])
y = df['Status_Yes'] 

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (7131970, 38)
X_test shape: (1782993, 38)
y_train shape: (7131970,)
y_test shape: (1782993,)


In [55]:
from xgboost import XGBClassifier

In [56]:
xgb_model = XGBClassifier().fit(X_train, y_train)

In [58]:
from sklearn.metrics import accuracy_score

In [59]:
y_pred = xgb_model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9991250666716022